In [1]:
import requests
import time
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

True

In [2]:
taxi_zone = pd.read_csv("taxi-zone-lookup.csv")
print (taxi_zone.head(10))

unique_zones = taxi_zone['Zone'].unique()
count_unique_zones = len(unique_zones)
print(f" il y a {count_unique_zones} zones")

unique_borough = taxi_zone['Borough'].unique()
count_unique_borough = len(unique_borough)
print(f" il y a {count_unique_borough} borough")

import re
# Appliquer le nettoyage sur chaque nom de zone avant '/'
taxi_zone['Zone'] = taxi_zone['Zone'].apply(lambda x: re.sub(r'/.*', '', x))

taxi_zone.head()

taxi_zone.describe(include='all')




   LocationID        Borough                     Zone
0           1            EWR           Newark Airport
1           2         Queens              Jamaica Bay
2           3          Bronx  Allerton/Pelham Gardens
3           4      Manhattan            Alphabet City
4           5  Staten Island            Arden Heights
5           6  Staten Island  Arrochar/Fort Wadsworth
6           7         Queens                  Astoria
7           8         Queens             Astoria Park
8           9         Queens               Auburndale
9          10         Queens             Baisley Park
 il y a 261 zones
 il y a 7 borough


,LocationID,Borough,Zone
count,265.000000,265,265
unique,NaN,7,255
top,NaN,Queens,Governor's Island
freq,NaN,69,3
mean,133.000000,NaN,NaN
std,76.643112,NaN,NaN
min,1.000000,NaN,NaN
25%,67.000000,NaN,NaN
50%,133.000000,NaN,NaN
75%,199.000000,NaN,NaN


test requete pour une ville 

In [3]:
import requests

# Paramètres pour la première ligne du DataFrame (LocationID = 1)
zone_name = "central Harlem"  # Zone associée à LocationID = 1

# URL de l'API Nominatim avec les paramètres de recherche
url = "https://nominatim.openstreetmap.org/search"

# Paramètres pour la requête
params = {
    'q': zone_name,  # Nom de la zone à rechercher
    'format': 'json',  # Format de la réponse JSON
    'addressdetails': 1,  # Inclure les détails de l'adresse
    'limit': 1  # Limiter à 1 résultat
}

# En-têtes pour imiter un véritable navigateur
headers = {
    "User-Agent": "MonApplication/1.0 (contact@monemail.com)",
    'Accept-Language': 'fr-FR,fr;q=0.9'  # Langue de préférence
}

# Faire la requête GET avec les paramètres et les en-têtes
response = requests.get(url, params=params, headers=headers)

# Extraire directement les coordonnées
data = response.json()
if data:  # Vérifier si des données sont présentes
    lat = data[0]['lat']
    lon = data[0]['lon']
    print(f"Latitude: {lat}, Longitude: {lon}")
else:
    print("Aucun résultat trouvé.")



Latitude: 22.9210997, Longitude: -83.1940368


Premiere passe O.S.M


In [4]:
import requests
import pandas as pd
import time

# Fonction pour récupérer la latitude et la longitude d'une zone
def get_lat_lon(taxi_zone):
    # URL de l'API Nominatim
    url = "https://nominatim.openstreetmap.org/search"

    # En-têtes pour imiter un véritable navigateur
    headers = {
        "User-Agent": "MonApplication/1.0 (contact@monemail.com)",
        'Accept-Language': 'fr-FR,fr;q=0.9'  # Langue de préférence
    }

    # Créer les colonnes 'lat' et 'lon' dans le DataFrame
    taxi_zone['lat'] = None
    taxi_zone['lon'] = None

    # Limiter la recherche à la zone de New York avec viewbox (bounding box)
    # Bounding box pour New York : (lat_min, lon_min, lat_max, lon_max)
    viewbox = '-77.371,38.303,-68.791,41.935'  # Limites géographiques de New York (bbox)

    # Boucle sur chaque ligne du DataFrame
    for index, row in taxi_zone.iterrows():
        zone_name = row['Zone']  # Récupérer le nom de la zone à partir de la colonne 'Zone'
        
        # Paramètres pour la requête
        params = {
            'q': zone_name,  # Nom de la zone à rechercher
            'format': 'json',  # Format de la réponse JSON
            'addressdetails': 1,
            'bounded' : 1,
            'viewbox' : viewbox,  # Inclure les détails de l'adresse
            'limit': 1  # Limiter à 1 résultat
        }

        # Faire la requête GET avec les paramètres et les en-têtes
        try:
            response = requests.get(url, params=params, headers=headers)
            response.raise_for_status()  # Vérifier si la requête a échoué (code 4xx ou 5xx)

            # Extraire directement les coordonnées
            data = response.json()
            if data:  # Vérifier si des données sont présentes
                lat = data[0]['lat']
                lon = data[0]['lon']
                taxi_zone.at[index, 'lat'] = lat  # Mettre à jour la colonne 'lat'
                taxi_zone.at[index, 'lon'] = lon  # Mettre à jour la colonne 'lon'
            else:
                print(f"Aucun résultat pour la zone: {zone_name}")
        
        except requests.exceptions.RequestException as e:
            print(f"Erreur pour la zone {zone_name}: {e}")
        
        # Ajouter un délai de 1 seconde avant la prochaine requête
        time.sleep(1)

    return taxi_zone

# Appeler la fonction pour récupérer la latitude et la longitude
taxi_zone = get_lat_lon(taxi_zone)

# Afficher le DataFrame mis à jour avec les nouvelles colonnes 'lat' et 'lon'
taxi_zone.head()


Aucun résultat pour la zone: Bensonhurst East
Aucun résultat pour la zone: Bensonhurst West
Aucun résultat pour la zone: Central Harlem
Aucun résultat pour la zone: Central Harlem North
Aucun résultat pour la zone: East Harlem North
Aucun résultat pour la zone: East Harlem South
Aucun résultat pour la zone: Greenwich Village North
Aucun résultat pour la zone: Lenox Hill West
Aucun résultat pour la zone: Lincoln Square East
Aucun résultat pour la zone: Lincoln Square West
Aucun résultat pour la zone: Ocean Parkway South
Aucun résultat pour la zone: Springfield Gardens North
Aucun résultat pour la zone: Springfield Gardens South
Aucun résultat pour la zone: Sunset Park East
Aucun résultat pour la zone: Sunset Park West
Aucun résultat pour la zone: Upper East Side North
Aucun résultat pour la zone: Upper East Side South
Aucun résultat pour la zone: Upper West Side North
Aucun résultat pour la zone: Upper West Side South
Aucun résultat pour la zone: Washington Heights North
Aucun résultat 

,LocationID,Borough,Zone,lat,lon
0,1,EWR,Newark Airport,40.6890640,-74.1772549
1,2,Queens,Jamaica Bay,40.6039936,-73.8354124
2,3,Bronx,Allerton,40.6112131,-74.8784999
3,4,Manhattan,Alphabet City,40.7251022,-73.9795833
4,5,Staten Island,Arden Heights,39.8103543,-75.4894662


In [5]:

# Identifier les zones avec des coordonnées manquantes (None)
missing_coords = taxi_zone[taxi_zone['lat'].isna()]
missing_coords

,LocationID,Borough,Zone,lat,lon
20,21,Brooklyn,Bensonhurst East,None,None
21,22,Brooklyn,Bensonhurst West,None,None
40,41,Manhattan,Central Harlem,None,None
41,42,Manhattan,Central Harlem North,None,None
73,74,Manhattan,East Harlem North,None,None
74,75,Manhattan,East Harlem South,None,None
112,113,Manhattan,Greenwich Village North,None,None
140,141,Manhattan,Lenox Hill West,None,None
141,142,Manhattan,Lincoln Square East,None,None
142,143,Manhattan,Lincoln Square West,None,None


Modification regex sur lattitude et longitude manquante 

2ème Passe O.S.M

In [6]:
# Liste des termes à enlever
terms_to_remove = ['central', 'north', 'south', 'east', 'west']

# Fonction de nettoyage modifiée
def clean_zone_name(zone_name):
    # Appliquer la première regex : supprimer les termes à enlever uniquement si le terme est en premier ou dernier mot
    for term in terms_to_remove:
        # Vérifier si le terme est au début ou à la fin du nom de la zone
        zone_name = re.sub(r'^\b' + term + r'\b', '', zone_name.lower())  # Si le terme est au début
        zone_name = re.sub(r'\b' + term + r'\b$', '', zone_name.lower())  # Si le terme est à la fin
    
    # Supprimer les espaces multiples ou les espaces en début et fin
    zone_name = re.sub(r'\s+', ' ', zone_name).strip()
    
    # Supprimer tous les caractères spéciaux entre parenthèses
    zone_name = re.sub(r'\([^)]*\)', '', zone_name)
    
    # Supprimer les caractères spéciaux
    zone_name = re.sub(r'[^a-zA-Z0-9\s]', '', zone_name)
    
    return zone_name

# Exemple d'application à la colonne 'Zone' de 'missing_coords'
missing_coords['Zone'] = missing_coords['Borough']+' '+ missing_coords['Zone'].apply(clean_zone_name)

# Afficher le DataFrame nettoyé
print(missing_coords)


     LocationID    Borough                          Zone   lat   lon
20           21   Brooklyn          Brooklyn bensonhurst  None  None
21           22   Brooklyn          Brooklyn bensonhurst  None  None
40           41  Manhattan              Manhattan harlem  None  None
41           42  Manhattan              Manhattan harlem  None  None
73           74  Manhattan              Manhattan harlem  None  None
74           75  Manhattan              Manhattan harlem  None  None
112         113  Manhattan   Manhattan greenwich village  None  None
140         141  Manhattan          Manhattan lenox hill  None  None
141         142  Manhattan      Manhattan lincoln square  None  None
142         143  Manhattan      Manhattan lincoln square  None  None
177         178   Brooklyn        Brooklyn ocean parkway  None  None
217         218     Queens    Queens springfield gardens  None  None
218         219     Queens    Queens springfield gardens  None  None
226         227   Brooklyn        

C:\Users\licor\AppData\Local\Temp\ipykernel_36516\3835607063.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_coords['Zone'] = missing_coords['Borough']+' '+ missing_coords['Zone'].apply(clean_zone_name)


In [7]:
# Appeler la fonction pour récupérer la latitude et la longitude
taxi_zone_2 = get_lat_lon(missing_coords)
taxi_zone_2.drop(columns=['Borough','Zone'])

C:\Users\licor\AppData\Local\Temp\ipykernel_36516\2971971538.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxi_zone['lat'] = None
C:\Users\licor\AppData\Local\Temp\ipykernel_36516\2971971538.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxi_zone['lon'] = None


,LocationID,lat,lon
20,21,40.6049772,-73.9934061
21,22,40.6049772,-73.9934061
40,41,40.8078578,-73.9454671
41,42,40.8078578,-73.9454671
73,74,40.8078578,-73.9454671
74,75,40.8078578,-73.9454671
112,113,40.7335844,-74.0028172
140,141,40.7664366,-73.9590168
141,142,40.7723194,-73.9844012
142,143,40.7723194,-73.9844012


In [8]:
# Fusionner les DataFrames 'taxi_zone' et 'taxi_zone_2' sur la clé 'LocationID' avec une jointure gauche
taxi_zone_clean = pd.merge(taxi_zone, taxi_zone_2[['LocationID', 'lat', 'lon']], on='LocationID', how='left')

# Remplacer les valeurs manquantes de 'lat' et 'lon' dans 'taxi_zone' avec celles de 'taxi_zone_2'
taxi_zone_clean['lat'] = taxi_zone_clean['lat_x'].combine_first(taxi_zone_clean['lat_y'])
taxi_zone_clean['lon'] = taxi_zone_clean['lon_x'].combine_first(taxi_zone_clean['lon_y'])

# Garder uniquement les colonnes d'origine de 'taxi_zone' (sans les colonnes suffixées '_x' et '_y')
taxi_zone_clean = taxi_zone_clean[taxi_zone.columns]

print(taxi_zone_clean.isna().sum())

taxi_zone_clean.head()


LocationID    0
Borough       0
Zone          0
lat           0
lon           0
dtype: int64


,LocationID,Borough,Zone,lat,lon
0,1,EWR,Newark Airport,40.6890640,-74.1772549
1,2,Queens,Jamaica Bay,40.6039936,-73.8354124
2,3,Bronx,Allerton,40.6112131,-74.8784999
3,4,Manhattan,Alphabet City,40.7251022,-73.9795833
4,5,Staten Island,Arden Heights,39.8103543,-75.4894662


In [9]:
taxi_zone_clean.describe(include='all')

,LocationID,Borough,Zone,lat,lon
count,265.000000,265,265,265,265
unique,NaN,7,255,243,243
top,NaN,Queens,Governor's Island,40.8078578,-73.9454671
freq,NaN,69,3,4,4
mean,133.000000,NaN,NaN,NaN,NaN
std,76.643112,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN
25%,67.000000,NaN,NaN,NaN,NaN
50%,133.000000,NaN,NaN,NaN,NaN
75%,199.000000,NaN,NaN,NaN,NaN


In [10]:
import plotly.express as px

# Créer un graphique de carte avec les données de latitude et de longitude
fig = px.scatter_geo(taxi_zone_clean, lat='lat', lon='lon', title='Localisation des Zones')

# Modifier la couleur des points en rouge
fig.update_traces(marker=dict(color='red'))

# Agrandir la carte en ajustant la taille de la figure
fig.update_layout(
    width=1200,  # Largeur en pixels
    height=800)  # Hauteur en pixels


# Afficher la carte
fig.show()


In [11]:
import folium
import webbrowser
import os
import time

# Créer une carte centrée sur New York (coordonnées de New York : latitude 40.7128, longitude -74.0060)
m = folium.Map(location=[40.7128, -74.0060], zoom_start=12)  # Zoom adapté pour New York

# Ajouter les points de ton DataFrame (taxi_zone_clean)
for index, row in taxi_zone_clean.iterrows():
    folium.Marker([row['lat'], row['lon']], popup=row['Zone'], icon=folium.Icon(color='red')).add_to(m)

# Sauvegarder la carte temporairement
map_path = "temp_map.html"
m.save(map_path)

# Ouvrir la carte dans le navigateur après une petite pause
time.sleep(1)  # On attend 1 seconde pour être sûr que le fichier est bien créé
webbrowser.open(f'file://{os.path.realpath(map_path)}', new=2)


True

In [12]:
import folium

# Créer une carte centrée sur New York
m = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Ajouter les points sur la carte
for index, row in taxi_zone_clean.iterrows():
    folium.CircleMarker([row['lat'], row['lon']], radius=5, color='red', fill=True, fill_color='red').add_to(m)

# Ajouter un titre à la carte
folium.Marker([40.7128, -74.0060], popup="New York").add_to(m)

# Sauvegarder la carte sous forme de fichier HTML et l'afficher dans le navigateur
m.save('map.html')

# Afficher la carte
m


In [13]:
taxi_zone_clean.to_csv("taxi-zone-lookup_gps.csv", encoding="utf-8", index=False)
